In [1]:
import multiSyncPy as msp
from sktime.utils.plotting import plot_correlations
from multiSyncPy import synchrony_metrics as sm
from scipy.signal import hilbert
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from data_functions import get_ant_datafiles_info, preprocess_antcolony_data
from multisync_functions import get_multisync_metrics, get_multisync_statistics
from IPython.display import display
from SurrogateGenerator import segm_shuffle, trajectory_plot, get_nth_colony, surrogate_aaft, get_n_aafts, get_n_segm_shuffles
import warnings

ant_colonies = get_ant_datafiles_info()
ant_colonies


,recording_location,file,location,n_ants,species_name,enterance
0,indoor,dataset/ants/IndoorDataset/Seq0001Object10Imag...,china,10,japanese_arched_ant,0
1,indoor,dataset/ants/IndoorDataset/Seq0002Object10Imag...,china,10,japanese_arched_ant,0
2,indoor,dataset/ants/IndoorDataset/Seq0003Object10Imag...,china,10,japanese_arched_ant,0
3,indoor,dataset/ants/IndoorDataset/Seq0004Object10Imag...,china,10,japanese_arched_ant,0
4,indoor,dataset/ants/IndoorDataset/Seq0005Object10Imag...,china,10,japanese_arched_ant,0
5,outdoor,dataset/ants/OutdoorDataset/Seq0006Object21Ima...,russia,73,carpenter_ant,1
6,outdoor,dataset/ants/OutdoorDataset/Seq0007Object26Ima...,greece,162,little_black_ant,1
7,outdoor,dataset/ants/OutdoorDataset/Seq0008Object31Ima...,russia,133,carpenter_ant,1
8,outdoor,dataset/ants/OutdoorDataset/Seq0009Object50Ima...,russia,193,carpenter_ant,1
9,outdoor,dataset/ants/OutdoorDataset/Seq0010Object30Ima...,usa,101,little_black_ant,0


In [5]:
warnings.filterwarnings("ignore")

example_surr_data = segm_shuffle(get_nth_colony(0), 'x', 'y','ant_id', 100)
ants = surrogate_aaft(example_surr_data, 'x', 'y', 'ant_id')
ant_dict = {ant_id: ants[ants['ant_id'] == ant_id] for ant_id in ants['ant_id'].unique()}
#print(f"Number of unique ants: {ants['ant_id'].nunique()}")

# make new columns 'coherence', 'csd', 'symbolic entropy', 'rho' in the ant_colonies dataframe
ant_colonies['coherence'] = None
ant_colonies['csd'] = None
ant_colonies['symbolic entropy'] = None
ant_colonies['rho'] = None

for i in range(10):
    example_surr_data = get_nth_colony(i)
    ants = surrogate_aaft(example_surr_data, 'x', 'y', 'ant_id')
    metrics_colony = get_multisync_metrics(ants, target_var='surrogate_x', n_observations_per_unit=100, id_column='ant_id')
    coherence, csd, symbolic_entropy, rho = metrics_colony
    #metrics.append([coherence, csd, symbolic_entropy, rho])

    # in row i of ant_colonies set the value of column 'coherence' to coherence, etc.
    ant_colonies.loc[i, 'coherence'] = coherence
    ant_colonies.loc[i, 'csd'] = csd
    ant_colonies.loc[i, 'symbolic entropy'] = symbolic_entropy
    ant_colonies.loc[i, 'rho'] = rho

display(ant_colonies.drop(columns=['file']))

original_metrics = ant_colonies.drop(columns=['file'])

,recording_location,location,n_ants,species_name,enterance,coherence,csd,symbolic entropy,rho
0,indoor,china,10,japanese_arched_ant,0,0.170392,0.182546,3.718252,0.995851
1,indoor,china,10,japanese_arched_ant,0,0.163746,0.168935,4.138792,0.997375
2,indoor,china,10,japanese_arched_ant,0,0.175648,0.171467,4.389467,0.994336
3,indoor,china,10,japanese_arched_ant,0,0.178599,0.209898,4.254928,0.997602
4,indoor,china,10,japanese_arched_ant,0,0.179775,0.146196,4.50813,0.999048
5,outdoor,russia,73,carpenter_ant,1,0.16959,0.182199,3.608036,0.946236
6,outdoor,greece,162,little_black_ant,1,0.155164,0.17124,3.747299,0.956951
7,outdoor,russia,133,carpenter_ant,1,0.168728,0.196974,3.797921,0.990915
8,outdoor,russia,193,carpenter_ant,1,0.185634,0.263996,3.228417,0.957732
9,outdoor,usa,101,little_black_ant,0,0.194262,0.399825,3.14276,0.959943


In [11]:
warnings.filterwarnings("ignore")

example_surr_data = segm_shuffle(get_nth_colony(0), 'x', 'y','ant_id', 1)
ants = surrogate_aaft(example_surr_data, 'x', 'y', 'ant_id')
ant_dict = {ant_id: ants[ants['ant_id'] == ant_id] for ant_id in ants['ant_id'].unique()}
#print(f"Number of unique ants: {ants['ant_id'].nunique()}")

# make new columns 'coherence', 'csd', 'symbolic entropy', 'rho' in the ant_colonies dataframe
ant_colonies['coherence'] = None
ant_colonies['csd'] = None
ant_colonies['symbolic entropy'] = None
ant_colonies['rho'] = None

for i in range(10):
    example_surr_data = segm_shuffle(get_nth_colony(i), 'x', 'y','ant_id', 1)
    ants = surrogate_aaft(example_surr_data, 'x', 'y', 'ant_id')
    metrics_colony = get_multisync_metrics(ants, target_var='surrogate_x', n_observations_per_unit=100, id_column='ant_id')
    coherence, csd, symbolic_entropy, rho = metrics_colony
    #metrics.append([coherence, csd, symbolic_entropy, rho])

    # in row i of ant_colonies set the value of column 'coherence' to coherence, etc.
    ant_colonies.loc[i, 'coherence'] = coherence
    ant_colonies.loc[i, 'csd'] = csd
    ant_colonies.loc[i, 'symbolic entropy'] = symbolic_entropy
    ant_colonies.loc[i, 'rho'] = rho

display(ant_colonies.drop(columns=['file']))

surr_metrics = ant_colonies.drop(columns=['file'])


,recording_location,location,n_ants,species_name,enterance,coherence,csd,symbolic entropy,rho
0,indoor,china,10,japanese_arched_ant,0,0.195806,0.158401,4.111066,0.999309
1,indoor,china,10,japanese_arched_ant,0,0.199401,0.176676,4.110373,0.999099
2,indoor,china,10,japanese_arched_ant,0,0.190493,0.227336,4.138099,0.999044
3,indoor,china,10,japanese_arched_ant,0,0.186743,0.191029,4.250547,0.999506
4,indoor,china,10,japanese_arched_ant,0,0.167436,0.155752,4.386761,0.999338
5,outdoor,russia,73,carpenter_ant,1,0.205801,0.224844,4.120838,0.938774
6,outdoor,greece,162,little_black_ant,1,0.166662,0.142178,4.1129,0.914738
7,outdoor,russia,133,carpenter_ant,1,0.172694,0.207612,3.850376,0.990191
8,outdoor,russia,193,carpenter_ant,1,0.194533,0.167403,3.984915,0.985246
9,outdoor,usa,101,little_black_ant,0,0.270501,0.473153,2.264995,0.963331


In [12]:
from scipy.stats import ttest_ind

metrics = ['coherence', 'csd', 'symbolic entropy', 'rho']

for metric in metrics:
    original_metric = pd.to_numeric(original_metrics[metric], errors='coerce')
    surr_metric = pd.to_numeric(surr_metrics[metric], errors='coerce')
    t_stat, p_val = ttest_ind(original_metric.dropna(), surr_metric.dropna())
    print(f"For metric {metric}, t-statistic: {t_stat}, p-value: {p_val}")

For metric coherence, t-statistic: -2.0727729462174427, p-value: 0.052827842596247176
For metric csd, t-statistic: -0.08104428304537403, p-value: 0.9363010719621094
For metric symbolic entropy, t-statistic: -0.33092988922897854, p-value: 0.7445187071777992
For metric rho, t-statistic: 0.06334176620680829, p-value: 0.9501924687730892


In [15]:
warnings.filterwarnings("ignore")

example_surr_data = segm_shuffle(get_nth_colony(0), 'x', 'y','ant_id', 10)
ants = surrogate_aaft(example_surr_data, 'x', 'y', 'ant_id')
ant_dict = {ant_id: ants[ants['ant_id'] == ant_id] for ant_id in ants['ant_id'].unique()}
#print(f"Number of unique ants: {ants['ant_id'].nunique()}")

# make new columns 'coherence', 'csd', 'symbolic entropy', 'rho' in the ant_colonies dataframe
ant_colonies['coherence'] = None
ant_colonies['csd'] = None
ant_colonies['symbolic entropy'] = None
ant_colonies['rho'] = None

for i in range(10):
    example_surr_data = segm_shuffle(get_nth_colony(i), 'x', 'y','ant_id', 10)
    ants = surrogate_aaft(example_surr_data, 'x', 'y', 'ant_id')
    metrics_colony = get_multisync_metrics(ants, target_var='surrogate_x', n_observations_per_unit=100, id_column='ant_id')
    coherence, csd, symbolic_entropy, rho = metrics_colony
    #metrics.append([coherence, csd, symbolic_entropy, rho])

    # in row i of ant_colonies set the value of column 'coherence' to coherence, etc.
    ant_colonies.loc[i, 'coherence'] = coherence
    ant_colonies.loc[i, 'csd'] = csd
    ant_colonies.loc[i, 'symbolic entropy'] = symbolic_entropy
    ant_colonies.loc[i, 'rho'] = rho

surr_metrics = ant_colonies.drop(columns=['file'])

from scipy.stats import ttest_ind

metrics = ['coherence', 'csd', 'symbolic entropy', 'rho']

for metric in metrics:
    original_metric = pd.to_numeric(original_metrics[metric], errors='coerce')
    surr_metric = pd.to_numeric(surr_metrics[metric], errors='coerce')
    t_stat, p_val = ttest_ind(original_metric.dropna(), surr_metric.dropna())
    print(f"For metric {metric}, t-statistic: {t_stat}, p-value: {p_val}")

For metric coherence, t-statistic: -1.6052386177091635, p-value: 0.12584223287127427
For metric csd, t-statistic: 0.44382309773076895, p-value: 0.6624596189852419
For metric symbolic entropy, t-statistic: -0.0011048600865025244, p-value: 0.999130602320587
For metric rho, t-statistic: 0.7791712388514853, p-value: 0.44600268589448233


In [16]:
warnings.filterwarnings("ignore")

example_surr_data = segm_shuffle(get_nth_colony(0), 'x', 'y','ant_id', 50)
ants = surrogate_aaft(example_surr_data, 'x', 'y', 'ant_id')
ant_dict = {ant_id: ants[ants['ant_id'] == ant_id] for ant_id in ants['ant_id'].unique()}
#print(f"Number of unique ants: {ants['ant_id'].nunique()}")

# make new columns 'coherence', 'csd', 'symbolic entropy', 'rho' in the ant_colonies dataframe
ant_colonies['coherence'] = None
ant_colonies['csd'] = None
ant_colonies['symbolic entropy'] = None
ant_colonies['rho'] = None

for i in range(10):
    example_surr_data = segm_shuffle(get_nth_colony(i), 'x', 'y','ant_id', 50)
    ants = surrogate_aaft(example_surr_data, 'x', 'y', 'ant_id')
    metrics_colony = get_multisync_metrics(ants, target_var='surrogate_x', n_observations_per_unit=100, id_column='ant_id')
    coherence, csd, symbolic_entropy, rho = metrics_colony
    #metrics.append([coherence, csd, symbolic_entropy, rho])

    # in row i of ant_colonies set the value of column 'coherence' to coherence, etc.
    ant_colonies.loc[i, 'coherence'] = coherence
    ant_colonies.loc[i, 'csd'] = csd
    ant_colonies.loc[i, 'symbolic entropy'] = symbolic_entropy
    ant_colonies.loc[i, 'rho'] = rho

surr_metrics = ant_colonies.drop(columns=['file'])

from scipy.stats import ttest_ind

metrics = ['coherence', 'csd', 'symbolic entropy', 'rho']

for metric in metrics:
    original_metric = pd.to_numeric(original_metrics[metric], errors='coerce')
    surr_metric = pd.to_numeric(surr_metrics[metric], errors='coerce')
    t_stat, p_val = ttest_ind(original_metric.dropna(), surr_metric.dropna())
    print(f"For metric {metric}, t-statistic: {t_stat}, p-value: {p_val}")

For metric coherence, t-statistic: -0.8854691374146435, p-value: 0.38757997115320364
For metric csd, t-statistic: 0.24208723812006522, p-value: 0.8114492793790847
For metric symbolic entropy, t-statistic: -0.6101529536131677, p-value: 0.5493869523559571
For metric rho, t-statistic: 0.040722223791078344, p-value: 0.9679656372213808


In [17]:
warnings.filterwarnings("ignore")

example_surr_data = segm_shuffle(get_nth_colony(0), 'x', 'y','ant_id', 100)
ants = surrogate_aaft(example_surr_data, 'x', 'y', 'ant_id')
ant_dict = {ant_id: ants[ants['ant_id'] == ant_id] for ant_id in ants['ant_id'].unique()}
#print(f"Number of unique ants: {ants['ant_id'].nunique()}")

# make new columns 'coherence', 'csd', 'symbolic entropy', 'rho' in the ant_colonies dataframe
ant_colonies['coherence'] = None
ant_colonies['csd'] = None
ant_colonies['symbolic entropy'] = None
ant_colonies['rho'] = None

for i in range(10):
    example_surr_data = segm_shuffle(get_nth_colony(i), 'x', 'y','ant_id', 100)
    ants = surrogate_aaft(example_surr_data, 'x', 'y', 'ant_id')
    metrics_colony = get_multisync_metrics(ants, target_var='surrogate_x', n_observations_per_unit=100, id_column='ant_id')
    coherence, csd, symbolic_entropy, rho = metrics_colony
    #metrics.append([coherence, csd, symbolic_entropy, rho])

    # in row i of ant_colonies set the value of column 'coherence' to coherence, etc.
    ant_colonies.loc[i, 'coherence'] = coherence
    ant_colonies.loc[i, 'csd'] = csd
    ant_colonies.loc[i, 'symbolic entropy'] = symbolic_entropy
    ant_colonies.loc[i, 'rho'] = rho

surr_metrics = ant_colonies.drop(columns=['file'])

from scipy.stats import ttest_ind

metrics = ['coherence', 'csd', 'symbolic entropy', 'rho']

for metric in metrics:
    original_metric = pd.to_numeric(original_metrics[metric], errors='coerce')
    surr_metric = pd.to_numeric(surr_metrics[metric], errors='coerce')
    t_stat, p_val = ttest_ind(original_metric.dropna(), surr_metric.dropna())
    print(f"For metric {metric}, t-statistic: {t_stat}, p-value: {p_val}")

For metric coherence, t-statistic: 1.0186751485596068, p-value: 0.32185462257193725
For metric csd, t-statistic: 0.8070434245437512, p-value: 0.43017701892737126
For metric symbolic entropy, t-statistic: -0.7152216491467425, p-value: 0.48364675546517655
For metric rho, t-statistic: -0.5714407910192818, p-value: 0.5747676549810656


In [18]:
warnings.filterwarnings("ignore")

example_surr_data = surrogate_aaft(get_nth_colony(i), 'x', 'y', 'ant_id') # segm_shuffle(get_nth_colony(0), 'x', 'y','ant_id', 100)
ants = surrogate_aaft(example_surr_data, 'surrogate_x', 'surrogate_y', 'ant_id')
ant_dict = {ant_id: ants[ants['ant_id'] == ant_id] for ant_id in ants['ant_id'].unique()}
#print(f"Number of unique ants: {ants['ant_id'].nunique()}")

# make new columns 'coherence', 'csd', 'symbolic entropy', 'rho' in the ant_colonies dataframe
ant_colonies['coherence'] = None
ant_colonies['csd'] = None
ant_colonies['symbolic entropy'] = None
ant_colonies['rho'] = None

for i in range(10):
    example_surr_data = surrogate_aaft(get_nth_colony(i), 'x', 'y', 'ant_id')
    ants = surrogate_aaft(example_surr_data, 'surrogate_x', 'surrogate_y', 'ant_id')
    metrics_colony = get_multisync_metrics(ants, target_var='surrogate_x', n_observations_per_unit=100, id_column='ant_id')
    coherence, csd, symbolic_entropy, rho = metrics_colony
    #metrics.append([coherence, csd, symbolic_entropy, rho])

    # in row i of ant_colonies set the value of column 'coherence' to coherence, etc.
    ant_colonies.loc[i, 'coherence'] = coherence
    ant_colonies.loc[i, 'csd'] = csd
    ant_colonies.loc[i, 'symbolic entropy'] = symbolic_entropy
    ant_colonies.loc[i, 'rho'] = rho

surr_metrics = ant_colonies.drop(columns=['file'])

from scipy.stats import ttest_ind

metrics = ['coherence', 'csd', 'symbolic entropy', 'rho']

for metric in metrics:
    original_metric = pd.to_numeric(original_metrics[metric], errors='coerce')
    surr_metric = pd.to_numeric(surr_metrics[metric], errors='coerce')
    t_stat, p_val = ttest_ind(original_metric.dropna(), surr_metric.dropna())
    print(f"For metric {metric}, t-statistic: {t_stat}, p-value: {p_val}")

For metric coherence, t-statistic: -0.035437552861762485, p-value: 0.9721208794092424
For metric csd, t-statistic: -0.06659886939792585, p-value: 0.9476352070224949
For metric symbolic entropy, t-statistic: 0.0, p-value: 1.0
For metric rho, t-statistic: -0.008790569372429273, p-value: 0.9930829265596596
